In [348]:
import glob
import docx
import re
import difflib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import collections
import os

In [220]:
pat = re.compile(r'[\-–,\?!\$%\;:\.\"]')

In [356]:
SEARCHING_HEADERS = set(['Proposal Title', 'Vertical(s)', 'Contributor(s) Name(s) and Title(s)',
                                                  'Disclosure of Any Conflict of Interests', 'Opportunity/Problem Being Solved',
                                                  'Impediments to Implementation', 'Recommended Actions',
                                                  'Program Type', 'Authority', 'Involved Agencies and/or Programs',
                                                  'Funding', 'Economic Benefits', 'Job Creation',
                                                  'Environmental Benefits', 'Equity Benefits',
                                                  'Stakeholder support/opposition', 'Key Experts'])
CSV_PROVIDED_HEADERS = set(['Proposal Title', 'Vertical(s)', 'Contributor(s) Name(s) and Title(s)',
                                                     'Authority', 'Funding', 'Involved Agencies and/or Programs'])
csv_headers = {'Proposal Title': 'Proposal Title', 'Vertical(s)': 'Verticals (max 3)',
              'Contributor(s) Name(s) and Title(s)': 'Contributor(s)',
              'Authority': 'Authority', 'Funding': 'Funding', 'Involved Agencies and/or Programs': 'Agency/Exec. Office'}
SEARCHING_HEADERS

{'Authority',
 'Contributor(s) Name(s) and Title(s)',
 'Disclosure of Any Conflict of Interests',
 'Economic Benefits',
 'Environmental Benefits',
 'Equity Benefits',
 'Funding',
 'Impediments to Implementation',
 'Involved Agencies and/or Programs',
 'Job Creation',
 'Key Experts',
 'Opportunity/Problem Being Solved',
 'Program Type',
 'Proposal Title',
 'Recommended Actions',
 'Stakeholder support/opposition',
 'Vertical(s)'}

In [147]:
filenames = glob.glob('ALREADY IN D BASE_/*.docx') + glob.glob('ALREADY IN D BASE_/*/*.docx')
filenames[:5]

['ALREADY IN D BASE_\\  Enactment of the “Energy Efficiency Acceleration Act of 2021” (the “EEAA”) .docx',
 'ALREADY IN D BASE_\\ Expansion of the State Energy Program (SEP).docx',
 'ALREADY IN D BASE_\\123 Agreements Through a Climate Lens.docx',
 'ALREADY IN D BASE_\\A Building Energy Performance Standard for Federal Buildings.docx',
 'ALREADY IN D BASE_\\A Just Transition for Coal Country_ Beyond Economic Recovery to Zero Carbon Economic Revival.docx']

In [148]:
len(filenames)

218

In [338]:
df = pd.read_csv('cleenProjectLibrary.csv')
df.head()

,Proposal Title,Contributor(s),Prof. Affiliations,Verticals (max 3),Authority,Funding,Misc.,Agency/Exec. Office,Full Proposal,Supporting documents
0,Budget Neutral Federal Fleet Emissions Reducti...,"Jeff Eckel,Guy Van Syckle",Hannon Armstrong,Auto & Transport,New legislation,No funding,Procurement,"DOE,GSA",10-14_ZEV-ESPCs.docx.pdf (https://dl.airtable....,NaN
1,"Sustainable Public Procurement: Mandates, Metr...","Steven Schooner,""Markus Speidel, Ph.D.""",George Washington University Law School,"Energy,Equity & EJ","Existing authority,Executive order,New legisla...",No funding,"WH Press,Procurement,Challenges/Awards","GSA,WH","""Sustainable Public Procurement_ Mandates, Met...",NaN
2,FERC Proceeding on System Peakiness,Richard Kauffman,Generate Capital,Energy,Existing authority,No funding,NaN,FERC,FERC Proceeding on System Peakiness_docx.pdf (...,NaN
3,Transform All Public Buildings by Reimagining ...,Jason Hartke,"International WELL Building Institute, ex-DOE",Buildings & Affordable Housing,Existing authority,No funding,"Fed/State Partners. ,Pub/Priv Partners.","DOE,GSA",Transform All Public Buildings by Reimagining ...,NaN
4,Expand federal STEM internships opportunities ...,"Anna Lising,Laurie Wiegand,Jeanette Pablo","Clean Energy for Biden, Utility Advantage, Ene...",Equity & EJ,Existing authority,Funding,NaN,DOE,"""Expand federal STEM internships opportunities...",NaN


In [347]:
sum(df['Full Proposal'].str.contains('123 Agreements Through a Climate Lens', case=False))

1

In [371]:
file = filenames[1]

def loose_match(header, search_list, diff=3):
    if len(header)<=1:
        return None
    for element in search_list:
        if count_diff(element.upper(), header.upper())<=3:
            return header
    return None

def get_information(filename, use_csv=True):
    information = collections.defaultdict(list)
    
    doc = docx.Document(filename)    
    proposal_name = os.path.splitext(os.path.basename(filename))[0]
    
    if use_csv and sum(df['Full Proposal'].str.contains(proposal_name))==1:
        row = df.loc[df['Full Proposal'].str.contains(proposal_name)]
        for header in CSV_PROVIDED_HEADERS:
            information[header] = row[csv_headers[header]].str.split(',')
        search_list = SEARCHING_HEADERS - CSV_PROVIDED_HEADERS
    else:
        search_list = SEARCHING_HEADERS
    
    current_header = None
    ignore = False
    for i, paragraph in enumerate(doc.paragraphs):
        val = paragraph.text
        val = val.replace('\t','').replace('\n','').strip()
        match = loose_match(val, search_list)
        #print(match)
        if match:
            current_header = match
            ignore = False
        else:
            csv_match = loose_match(val, CSV_PROVIDED_HEADERS)
            if use_csv and csv_match:
                current_header = csv_match
                ignore = True
            if current_header and not ignore:
                information[current_header].append(val)
    print('Missing headers:', set(SEARCHING_HEADERS)-set(information.keys()))
    return information

information = get_information(file, use_csv=True)
information

Missing headers: {'Contributor(s) Name(s) and Title(s)', 'Disclosure of Any Conflict of Interests', 'Vertical(s)', 'Recommended Actions', 'Proposal Title', 'Stakeholder support/opposition'}


defaultdict(list,
            {'PROPOSAL TITLE': ['Expansion of the State Energy Program (SEP)'],
             'Verticals': ['Buildings, energy, environment, affordable housing, infrastructure, transportation, innovation, agriculture, circular economy, consumer products, manufacturing, environmental justice, and state-local-federal interface.',
              'Contributor',
              'Jeff Genzer, Counsel, NASEO',
              'Conflict of Interest',
              'I have been counsel to NASEO since the organization began in 1986.  NASEO represents the state energy offices in all the states, territories and Washington, DC.  NASEO is committed to representing the views of the states in Washington, DC and to assist the states in developing energy policies and programs that assist the economy, promote energy efficiency, renewable energy, respond to climate change, prepare and respond to energy emergencies, address cyber and physical security and assist in economic development.'],
    

In [298]:
paragraph = doc.paragraphs[0]
print(paragraph.runs[0].text)
print(paragraph.runs[0].bold)
print(paragraph.style.font.highlight_color)

Proposal Title	
True
None


In [258]:
header_counts = collections.defaultdict(int) # count how many times each unique header comes up
for file in filenames:
    try:
        doc = docx.Document(file)
        for paragraph in doc.paragraphs:
            text = paragraph.text.replace('\n','').replace('\t','').replace(':','')
            if len(paragraph.runs)>0:
                if paragraph.runs[0].bold:
                    header_counts[text.upper().strip()] += 1
    except:
        continue
        
print(len(header_counts))
header_counts

312


defaultdict(int,
            {'PROPOSAL TITLE': 200,
             'VERTICALS': 203,
             'CONTRIBUTOR(S) NAME(S) & TITLE(S)': 200,
             'DISCLOSURE OF ANY CONFLICT OF INTEREST': 196,
             'OPPORTUNITY/PROBLEM BEING SOLVED': 182,
             'IMPEDIMENTS TO IMPLEMENTATION': 199,
             'RECOMMENDED ACTIONS': 203,
             'PROGRAM TYPE': 198,
             'AUTHORITY': 201,
             'INVOLVED AGENCIES AND/OR PROGRAMS': 204,
             'FUNDING': 195,
             'ECONOMIC BENEFITS': 199,
             'JOB CREATION': 196,
             'ENVIRONMENTAL BENEFITS': 200,
             'EQUITY BENEFITS': 198,
             'STAKEHOLDER SUPPORT AND OPPOSITION': 166,
             'KEY EXPERTS': 197,
             'REFERENCE MATERIALS': 173,
             'CONTRIBUTOR': 1,
             'CONFLICT OF INTEREST': 1,
             'TITLE': 2,
             'DISCLOSURE OF ANY CONFLICT OF INTEREST NO CONFLICTS': 1,
             'STAKEHOLDER SUPPORT AND OPPOSITION UNDETE

In [223]:
def count_diff(header, val):
    count = 0
    for s in difflib.ndiff(header, val):
        if s[0]==' ': continue
        elif s[0]=='-':
            count+=1
        elif s[0]=='+':
            count+=1
    return count

In [207]:
def get_differences(doc, header):
    differences = {}
    for paragraph in doc.paragraphs:
        val = paragraph.text
        val = val.replace('\t','').replace('\n','')
        if not re.search(pat, val) and len(val)>1:
            differences[val] = count_diff(header.upper(), val.upper())
    return differences

def get_header(doc, header):
    diffs = get_differences(doc, header)
    return list(diffs.keys())[np.argmin(list(diffs.values()))]

def get_header_index(doc, header):
    diffs = get_differences(doc, header)
    return np.argmin(list(diffs.values()))

In [173]:
def filter_out_bad_headers(doc):
    real_headers = []
    for search_header in SEARCHING_HEADERS:
        real_headers.append(get_header(doc, search_header))
    return real_headers

filter_out_bad_headers(docx.Document(filenames[100])) 

['Proposal Title',
 'Verticals',
 'Contributor(s) Name(s) & Title(s)',
 'Disclosure of Any Conflict of Interest',
 'Opportunity/Problem Being Solved',
 'Impediments to Implementation',
 'Recommended Actions',
 'Program Type',
 'Authority',
 'Involved Agencies and/or Programs',
 'Funding',
 'Economic Benefits',
 'Job Creation',
 'Environmental Benefits',
 'Equity Benefits',
 'Stakeholder Support and Opposition',
 'Key Experts']